In [4]:
import requests
for method in ("get","post","put","delete","head","options","patch"):
    original = getattr(requests,method)

    def insecure_request(*args, _original = original, **kwargs):
        kwargs["verify"] = False
        return _original(*args,**kwargs)
    
    setattr(requests,method,insecure_request)

In [14]:
import json

# Full path to your JSON file
file_path = r"C:\Users\GenaiblrpioUsr2\Desktop\Team24\database\data\jsons\synthetic_data.json"

# Open and load JSON
with open(file_path, "r", encoding="utf-8") as f:
    trades = json.load(f)


Loaded 37 trades
T100001 AAPL 189.52
T100002 TSLA 230.18
T100003 GOOGL 129.24
T100004 AMZN 174.11
T100005 MSFT 402.17
T100006 IBM 142.55
T100007 META 488.12
T100008 NFLX 573.33
T100009 NVDA 875.21
T100010 AMD 201.12
T100011 INTC 43.9
T100012 BRK-B 413.25
T100013 SPY 520.44
T100014 QQQ 432.11
T100015 GLD 198.55
T100016 TLT 101.2
T100017 10Y Treasury 99.75
T100018 EUR/USD FUT 1.0824
T100019 GBP/USD FUT 1.2666
T100020 USO 70.12
T100021 BND 76.33
T100022 VTI 261.4
T100023 EFA 81.22
T100024 EEM 41.5
T100025 XOM 119.7
T100026 CVX 152.45
T100027 BABA 78.33
T100028 ORCL 123.6
T100029 ADBE 612.9
T100030 CRM 289.11
T100031 JPM 189.1
T100032 BAC 41.75
T100033 WMT 62.11
T100034 KO 59.5
T100035 PEP 172.44
T100036 ABNB 149.12
T100037 SHOP 89.5


In [ ]:
fastapi
uvicorn
langchain
langchain-openai
langgraph
langchain-community
chromadb
fpdf
pydantic
python-dotenv


In [ ]:
[
  {
    "trade_id": "T982134",
    "instrument": "AAPL",
    "isin": "US0378331005",
    "trade_date": "2025-02-05",
    "settlement_date": "2025-02-07",
    "buyer_lei": "5493001KJTIIGC8Y1R12",
    "seller_lei": "213800D1EI4B8LMZ6L14",
    "price": 186.20,
    "quantity": 200,
    "trade_type": "BUY",
    "venue": "NASDAQ"
  },
  {
    "trade_id": "T982135",
    "instrument": "TSLA",
    "isin": "US88160R1014",
    "trade_date": "2025-02-05",
    "settlement_date": "2025-02-08",
    "buyer_lei": "213800D1EI4B8LMZ6L14",
    "seller_lei": "7ZW0H3557EJ8EZVQY350",
    "price": 225.10,
    "quantity": 100,
    "trade_type": "SELL",
    "venue": "NYSE"
  }
]


In [ ]:
MiFID II Article 26 - Transaction Reporting Requirements:

Investment firms shall report complete and accurate details of transactions no later than the close of the following working day (T+1).

The report must contain:
 - Buyer's LEI
 - Seller's LEI
 - Instrument Identifier (ISIN)
 - Price and Quantity
 - Trading Venue
 - Trade Date and Settlement Date

Article 4:
Instruments must be reported using valid ISIN formats and be traceable to market identifiers.

Article 12:
Any missing LEI, invalid ISIN, or inconsistent trade details should be flagged as compliance anomalies.


In [ ]:
import datetime

def log(message):
    ts = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    print(f"[{ts}] {message}")


In [ ]:
from fpdf import FPDF
import os
from config import OUTPUT_DIR

def generate_pdf(report_data, filename="compliance_report.pdf"):
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=12)

    pdf.cell(180, 10, txt="Regulatory Compliance Report", ln=1)

    pdf.set_font("Arial", size=10)
    for section in report_data:
        pdf.multi_cell(0, 8, section)
        pdf.ln(4)

    output_path = os.path.join(OUTPUT_DIR, filename)
    pdf.output(output_path)

    return output_path


In [ ]:
import os
from config import OUTPUT_DIR

def generate_xbrl(data, filename="report.xbrl"):
    xbrl = "<xbrl>\n"
    for section in data:
        for k, v in section.items():
            tag = k.replace(" ", "_")
            xbrl += f"  <{tag}>{v}</{tag}>\n"
    xbrl += "</xbrl>"

    output_path = os.path.join(OUTPUT_DIR, filename)
    with open(output_path, "w") as f:
        f.write(xbrl)

    return output_path


In [ ]:
import os
from config import OUTPUT_DIR

def generate_xbrl(data, filename="report.xbrl"):
    xbrl = "<xbrl>\n"
    for section in data:
        for k, v in section.items():
            tag = k.replace(" ", "_")
            xbrl += f"  <{tag}>{v}</{tag}>\n"
    xbrl += "</xbrl>"

    output_path = os.path.join(OUTPUT_DIR, filename)
    with open(output_path, "w") as f:
        f.write(xbrl)

    return output_path


In [ ]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
from utils.logger import log

def regulatory_agent(state):
    log("Running Regulatory Interpretation Agent...")

    vectordb = state["vectordb"]
    transactions = state["clean_transactions"]

    llm = ChatOpenAI(model="gpt-4o-mini")

    qa = RetrievalQA.from_chain_type(
        llm=llm,
        retriever=vectordb.as_retriever(),
        chain_type="stuff"
    )

    rule_map = []

    for t in transactions:
        query = f"Which MiFID II compliance rules apply to this transaction? {t}"
        answer = qa.run(query)
        rule_map.append({
            "trade_id": t["trade_id"],
            "rules": answer
        })

    state["rule_map"] = rule_map

    log("Regulatory interpretation completed.")

    return state


In [ ]:
from langchain_openai import ChatOpenAI
from utils.logger import log

def analyzer_agent(state):
    log("Running Compliance Analyzer Agent...")

    llm = ChatOpenAI(model="gpt-4o-mini")

    transactions = state["clean_transactions"]
    rule_map = state["rule_map"]

    results = []

    for t, r in zip(transactions, rule_map):
        query = f"""
        Evaluate this transaction against the given compliance rules.
        Transaction: {t}
        Rules: {r['rules']}
        Identify anomalies, delays, missing identifiers, or inconsistencies.
        Return a detailed compliance assessment.
        """
        assessment = llm.predict(query)

        results.append({
            "trade_id": t["trade_id"],
            "assessment": assessment
        })

    state["analysis"] = results

    log("Compliance analysis completed.")

    return state


In [ ]:
from utils.pdf_generator import generate_pdf
from utils.xbrl_generator import generate_xbrl
from utils.logger import log

def report_agent(state):
    log("Running Report Generator Agent...")

    analysis = state["analysis"]

    # Format PDF narrative
    pdf_sections = [
        f"Trade {a['trade_id']} Assessment:\n{a['assessment']}\n"
        for a in analysis
    ]

    pdf_path = generate_pdf(pdf_sections)

    # Prepare XBRL-structured data
    xbrl_data = [
        {"trade_id": a["trade_id"], "assessment": a["assessment"]}
        for a in analysis
    ]

    xbrl_path = generate_xbrl(xbrl_data)

    state["report_pdf"] = pdf_path
    state["report_xbrl"] = xbrl_path

    log("Reports generated successfully.")

    return state


In [ ]:
from langgraph.graph import Graph
from agents.ingestion_agent import ingestion_agent
from agents.regulatory_agent import regulatory_agent
from agents.analyzer_agent import analyzer_agent
from agents.report_agent import report_agent

workflow = Graph()

workflow.add_node("ingestion", ingestion_agent)
workflow.add_node("regulatory", regulatory_agent)
workflow.add_node("analyzer", analyzer_agent)
workflow.add_node("reporter", report_agent)

workflow.add_edge("ingestion", "regulatory")
workflow.add_edge("regulatory", "analyzer")
workflow.add_edge("analyzer", "reporter")

workflow.set_entry_point("ingestion")

graph_app = workflow.compile()


In [ ]:
from fastapi import FastAPI, UploadFile
import json
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

from graph.workflow_graph import graph_app
from config import REG_DOCS_DIR, VECTOR_DB_DIR
from utils.logger import log

app = FastAPI()

# --- Build Vector DB ---
def initialize_vectordb():
    log("Initializing Vector DB...")
    
    text = open(f"{REG_DOCS_DIR}/mifid2.txt").read()

    splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=200)
    docs = splitter.split_text(text)

    vectordb = Chroma.from_texts(
        docs,
        OpenAIEmbeddings(),
        persist_directory=VECTOR_DB_DIR,
        collection_name="reg_rules"
    )
    return vectordb


vectordb = initialize_vectordb()


# --- API Endpoint ---
@app.post("/process")
async def process_file(file: UploadFile):

    data = json.loads(await file.read())

    initial_state = {
        "transactions": data,
        "vectordb": vectordb
    }

    output = graph_app.invoke(initial_state)

    return {
        "status": "completed",
        "pdf_report": output["report_pdf"],
        "xbrl_report": output["report_xbrl"],
        "analysis": output["analysis"]
    }


In [ ]:
uvicorn app:app --reload
